In [20]:
import pandas as pd
dataset = pd.read_csv('C:/Users/poom/Documents/Sorted_Poom/KU/4_2/Project/Data/ptt.csv', header=0, infer_datetime_format=True, parse_dates=['Date'], index_col=['Date'])
df_real = dataset
dataset = dataset.drop(index = dataset[dataset['Close']>100].index)
dataset = dataset.drop(index = dataset[dataset['Close'].isnull()].index)
dataset.head()

,Close,Volume
Date,,
2008-10-29,14.000000,117061000
2008-10-30,15.300000,149869000
2008-10-31,15.900000,101556000
2008-11-03,17.700001,116285000
2008-11-04,18.299999,85938000


In [21]:
from sklearn.preprocessing import MinMaxScaler

In [28]:
scaler = MinMaxScaler(feature_range=(0, 1))
dataset = pd.DataFrame(scaler.fit_transform(dataset.values),index=dataset.index,columns=dataset.columns)
split_date = '2018-01-01'
train = dataset.loc[:split_date].values.reshape(-1, len(dataset.columns))
test = dataset.loc[split_date:].values.reshape(-1, len(dataset.columns))

In [29]:
df_scaled = pd.DataFrame(scaled,index=dataset.index,columns=dataset.columns)
df_scaled

,Close,Volume
Date,,
2008-10-29,0.006652,0.351882
2008-10-30,0.035477,0.450502
2008-10-31,0.048780,0.305275
2008-11-03,0.088692,0.349550
2008-11-04,0.101996,0.258327
2008-11-05,0.101996,0.363272
2008-11-06,0.101996,0.317121
2008-11-07,0.099778,0.366239
2008-11-10,0.095344,0.255054


In [31]:
import numpy as np

n_input =5
n_out=5

def lstm_reshape(dataA, dimension):
        dataA = np.reshape(dataA, (dataA.shape[0], dataA.shape[1], dimension))
        return dataA

def lstm_reshape_y(dataA, dimension):
        dataA = np.reshape(dataA, (dataA.shape[0], dimension))
        return dataA

# convert history into inputs and outputs
def to_supervised(train, n_input, n_out):
        features_set = []  
        labels = []  
        for i in range(n_input, len(train)-n_out + 1):  
            features_set.append(train[i-n_input:i])
            labels.append(train[i:i+n_out,0])
        return lstm_reshape(np.array(features_set),train.shape[1]), lstm_reshape_y(np.array(labels),n_out)

train_x, train_y = to_supervised(train, n_input, n_out)
test_x, test_y = to_supervised(test, n_input, n_out)
n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]

In [33]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras.layers import LSTM
import time
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

lstm_model = Sequential()
lstm_model.add(Dense(n_timesteps, input_shape=(n_timesteps, n_features), activation='relu'))
lstm_model.add(LSTM(1024, activation='relu'))
lstm_model.add(Dense(n_outputs, activation='linear'))
lstm_model.summary()

lstm_model.compile(Adam(lr=0.0001), loss='mean_squared_error', metrics=['mse'])
early_stop = EarlyStopping(monitor='val_mean_squared_error', patience=100, verbose=2, restore_best_weights=True)
t_lstm = time.time()
history_lstm_model = lstm_model.fit(train_x, train_y , validation_split=0.05, epochs=5, batch_size=256, verbose=2, shuffle=False, callbacks=[early_stop])
fin_time_lstm = time.time() - t_lstm

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 5, 5)              15        
_________________________________________________________________
lstm_2 (LSTM)                (None, 1024)              4218880   
_________________________________________________________________
dense_4 (Dense)              (None, 5)                 5125      
Total params: 4,224,020
Trainable params: 4,224,020
Non-trainable params: 0
_________________________________________________________________
Train on 2115 samples, validate on 112 samples
Epoch 1/5
 - 11s - loss: 0.1512 - mean_squared_error: 0.1512 - val_loss: 0.3576 - val_mean_squared_error: 0.3576
Epoch 2/5
 - 5s - loss: 0.1458 - mean_squared_error: 0.1458 - val_loss: 0.3464 - val_mean_squared_error: 0.3464
Epoch 3/5
 - 5s - loss: 0.1373 - mean_squared_error: 0.1373 - val_loss: 0.3280 - val_mean_squared_error: 0.3280
Epoch 4/5
 -

In [34]:
print('Total Training Time =',fin_time_lstm)

Total Training Time = 34.052162647247314


In [35]:
ind_Pred = list(range(0,len(dataset['Close'])-(n_input+n_out),n_out))
ind_Pred

[0,
 5,
 10,
 15,
 20,
 25,
 30,
 35,
 40,
 45,
 50,
 55,
 60,
 65,
 70,
 75,
 80,
 85,
 90,
 95,
 100,
 105,
 110,
 115,
 120,
 125,
 130,
 135,
 140,
 145,
 150,
 155,
 160,
 165,
 170,
 175,
 180,
 185,
 190,
 195,
 200,
 205,
 210,
 215,
 220,
 225,
 230,
 235,
 240,
 245,
 250,
 255,
 260,
 265,
 270,
 275,
 280,
 285,
 290,
 295,
 300,
 305,
 310,
 315,
 320,
 325,
 330,
 335,
 340,
 345,
 350,
 355,
 360,
 365,
 370,
 375,
 380,
 385,
 390,
 395,
 400,
 405,
 410,
 415,
 420,
 425,
 430,
 435,
 440,
 445,
 450,
 455,
 460,
 465,
 470,
 475,
 480,
 485,
 490,
 495,
 500,
 505,
 510,
 515,
 520,
 525,
 530,
 535,
 540,
 545,
 550,
 555,
 560,
 565,
 570,
 575,
 580,
 585,
 590,
 595,
 600,
 605,
 610,
 615,
 620,
 625,
 630,
 635,
 640,
 645,
 650,
 655,
 660,
 665,
 670,
 675,
 680,
 685,
 690,
 695,
 700,
 705,
 710,
 715,
 720,
 725,
 730,
 735,
 740,
 745,
 750,
 755,
 760,
 765,
 770,
 775,
 780,
 785,
 790,
 795,
 800,
 805,
 810,
 815,
 820,
 825,
 830,
 835,
 840,
 845,
 8

In [39]:
remain = len(dataset['Close']) % n_input
remain

2

In [40]:
Y_true = df_real['Close'][n_input:-remain];Y_true

Date
2008-11-05    18.299999
2008-11-06    18.299999
2008-11-07    18.200001
2008-11-10    18.000000
2008-11-11    17.100000
2008-11-12    16.700001
2008-11-13    16.600000
2008-11-14    16.400000
2008-11-17    16.799999
2008-11-18    15.900000
2008-11-19    15.000000
2008-11-20    14.200000
2008-11-21    14.100000
2008-11-24    13.700000
2008-11-25    13.800000
2008-11-26    14.300000
2008-11-27    14.000000
2008-11-28    14.700000
2008-12-01    14.400000
2008-12-02    14.300000
2008-12-03    14.400000
2008-12-04    14.400000
2008-12-08    15.400000
2008-12-09    16.799999
2008-12-11    17.200001
2008-12-12    17.200001
2008-12-15    17.700001
2008-12-16    18.000000
2008-12-17    17.700001
2008-12-18    18.000000
                ...    
2019-08-06    45.500000
2019-08-07    45.000000
2019-08-08    44.250000
2019-08-09    44.250000
2019-08-13    43.000000
2019-08-14    43.250000
2019-08-15    42.250000
2019-08-16    42.750000
2019-08-19    43.250000
2019-08-20    42.750000
2019-08-21 

In [48]:
ind_Pred[-1]

2645

In [54]:
all_x[ind_Pred]

array([[[0.00665188, 0.35188219],
        [0.03547672, 0.45050215],
        [0.04878049, 0.30527458],
        [0.08869182, 0.34954955],
        [0.10199555, 0.2583273 ]],

       [[0.10199555, 0.36327182],
        [0.10199555, 0.31712112],
        [0.09977829, 0.36623872],
        [0.09534368, 0.25505379],
        [0.07538803, 0.18524608]],

       [[0.06651887, 0.17923714],
        [0.06430155, 0.18117299],
        [0.05986696, 0.1543537 ],
        [0.06873612, 0.07656213],
        [0.04878049, 0.13479083]],

       ...,

       [[0.64966742, 0.08952028],
        [0.63303771, 0.25134773],
        [0.63858095, 0.06196182],
        [0.60532152, 0.2654244 ],
        [0.59977828, 0.24098674]],

       [[0.60532152, 0.08503176],
        [0.65521066, 0.32976454],
        [0.67738361, 0.28271025],
        [0.66075389, 0.14613297],
        [0.66075389, 0.13728158]],

       [[0.67738361, 0.15540459],
        [0.68292684, 0.0881249 ],
        [0.69401332, 0.09948087],
        [0.70509979, 0.13

In [58]:
all_x ,all_y =to_supervised(dataset.values.reshape(-1, len(dataset.columns))[:-remain], n_input, n_out)
Y_pred = lstm_model.predict(all_x[ind_Pred])

In [59]:
Y_hat = []
for i in range(len(Y_pred)):
    for j in range(len(Y_pred[i])):
        Y_hat.append(Y_pred[i][j])

In [69]:
df_dummy = pd.DataFrame(np.append(df_real['Close'].values,df_real['Volume'].values))
scaling_dummy = scaler.fit_transform(df_dummy)
Y_hat_true = scaler.inverse_transform(np.array(Y_hat).reshape(-1, 1))

In [70]:
Y_hat_true

array([[38200276.],
       [36906040.],
       [39947652.],
       ...,
       [39554372.],
       [31239680.],
       [35984150.]], dtype=float32)